In [1]:
from pymatgen import MPRester, Element, Composition
from pymatgen.phasediagram.maker import PhaseDiagram
from pymatgen.phasediagram.entries import PDEntry
from pymatgen.phasediagram.analyzer import PDAnalyzer
from pymatgen.analysis.reaction_calculator import ComputedReaction
import matplotlib.pyplot as plt
import sys

In [3]:
def get_chempots_phases_enthalpies(chsys, formula_id, elem):
    chempots = []
    evolutions = []
    phases = []
    enthalpies = []
    yvec = []
    
    with MPRester() as m:
        entries = m.get_entries_in_chemsys(chsys)
        compound = m.get_entry_by_material_id(formula_id)
    
    pd = PhaseDiagram(entries)
    pda = PDAnalyzer(pd)
    evolution_profile = pda.get_element_profile(Element(elem), Composition(compound.name))
    ref = evolution_profile[0]['element_reference'].energy_per_atom

    reac = [compound]
    reac.append(evolution_profile[0]['element_reference'])
    
    for stage in evolution_profile:
        chempots.append(-(stage['chempot'] - ref))
        evolutions.append(stage['evolution'])
        namelist = []
        for entry in stage['entries']:
            namelist.append(entry.name)
        phases.append(namelist)
    
    yshift = 0
    for stage in evolution_profile:
        rxn = ComputedReaction(reac, stage['entries']) #calculate reaction enthalpy
        rxn.normalize_to(Composition(compound.name))
        enthalpies.append(rxn.calculated_reaction_energy) 
        if abs(stage['evolution']) < 0.0001: #find y-axis shift (force all compounds to be stable)
            yshift = stage['evolution']*stage['chempot'] + rxn.calculated_reaction_energy
    for i in range(len(chempots)): #manipulate and shift data for graphs
        ynew = (evolutions[i]*chempots[i] + enthalpies[i]) - yshift
        yvec.append(ynew)            
    
    return [chempots, phases, yvec, enthalpies]

In [4]:
LCO_cpe = get_chempots_phases_enthalpies(['Li','Co','O'],'mp-24850','O')
LFP_cpe = get_chempots_phases_enthalpies(['Li','Fe','P','O'],'mp-19017','O')
NMC_cpe = get_chempots_phases_enthalpies(['Li','Ni','Mn','Co','O'],'mp-762156','O')

In [5]:
import pandas as pnds
pnds.options.display.float_format = '{:.3f}'.format

T_1 = pnds.DataFrame({'Chemical Potential':LCO_cpe[0],
                      'Phase Equilibria':LCO_cpe[1],
                     'Reaction Enthalpy':LCO_cpe[3]})

T_2 = pnds.DataFrame({'Chemical Potential':LFP_cpe[0],
                      'Phase Equilibria':LFP_cpe[1],
                     'Reaction Enthalpy':LFP_cpe[3]})

T_3 = pnds.DataFrame({'Chemical Potential':NMC_cpe[0],
                      'Phase Equilibria':NMC_cpe[1],
                     'Reaction Enthalpy':NMC_cpe[3],
                     'yvec':NMC_cpe[2]})
display(T_1)
display(T_2)
display(T_3)

,Chemical Potential,Phase Equilibria,Reaction Enthalpy
0,-0.000,"[Li7Co5O12, Li(CoO2)2]",-0.084
1,0.378,[LiCoO2],-0.000
2,2.644,"[Li6CoO4, CoO]",1.322
3,2.660,"[Co, Li6CoO4]",3.539
4,2.903,"[Co, Li2O]",4.023
5,6.213,"[Co, Li]",7.129


,Chemical Potential,Phase Equilibria,Reaction Enthalpy
0,-0.000,"[Li3Fe2(PO4)3, Fe2O3]",-0.858
1,1.716,[LiFePO4],0.000
2,3.608,"[FeP, Fe2P, Li3PO4]",9.621
3,4.663,"[Li2Fe12P7, Li3PO4, FeP]",10.352
4,4.704,[LiFeP],15.887


,Chemical Potential,Phase Equilibria,Reaction Enthalpy,yvec
0,-0.000,"[LiCoNiO4, Li(CoO2)2, Li2Mn3NiO8]",-1.015,0.000
1,0.277,"[Li2Mn3NiO8, NiO, Li(CoO2)2, Li2MnO3]",-0.807,-0.000
2,0.500,"[Li2Mn3NiO8, NiO, Li(CoO2)2, LiCoO2]",-0.682,-0.167
3,0.679,"[Li2Mn3NiO8, NiO, Li2MnCo3O8, LiCoO2]",-0.547,-0.346
4,0.753,"[Li2Mn3NiO8, NiO, LiCoO2, Co3O4]",-0.196,-0.434
5,1.422,"[Li2Mn3NiO8, NiO, Co3O4, Li2MnO3]",0.279,-1.550
6,1.692,"[Li2MnO3, NiO, Co3O4, Mn3O4]",1.407,-2.090
7,1.868,"[Li2MnO3, Ni, Co3O4, Mn3O4]",5.144,-2.561
8,1.940,"[Li2MnO3, Ni, Mn3O4, CoO]",7.084,-2.895
9,1.988,"[LiMnO2, Ni, CoO, Li2MnO3]",7.746,-3.165


In [6]:
LLZO_cpe = get_chempots_phases_enthalpies(['Li','La','Zr','O'],'mp-942733','O')
LTP_cpe = get_chempots_phases_enthalpies(['Li','Ti','P','O'],'mp-18640','O')
LPS_cpe = get_chempots_phases_enthalpies(['Li','P','S', 'O'],'mp-985583','O')

T_3 = pnds.DataFrame({'Chemical Potential':LPS_cpe[0],
                      'Phase Equilibria':LPS_cpe[1],
                     'Reaction Enthalpy':LPS_cpe[3]})

T_4 = pnds.DataFrame({'Chemical Potential':LLZO_cpe[0],
                      'Phase Equilibria':LLZO_cpe[1],
                     'Reaction Enthalpy':LLZO_cpe[3]})

T_5 = pnds.DataFrame({'Chemical Potential':LTP_cpe[0],
                      'Phase Equilibria':LTP_cpe[1],
                     'Reaction Enthalpy':LTP_cpe[3]})
display(T_3)
display(T_4)
display(T_5)

,Chemical Potential,Phase Equilibria,Reaction Enthalpy
0,-0.000,"[Li2S2O7, P2O5, SO3]",-42.358
1,1.771,"[Li2S2O7, SO2, P2O5]",-40.587
2,2.012,"[Li2S2O7, SO2, LiPO3]",-38.575
3,2.339,"[Li2S2O7, S8O, LiPO3]",-29.803
4,2.340,"[Li2SO4, S8O, LiPO3]",-23.076
5,2.572,"[Li4P2O7, S8O, Li2SO4]",-19.378
6,2.658,"[S8O, Li3PO4]",-15.558
7,3.457,[Li3PS4],0.000


,Chemical Potential,Phase Equilibria,Reaction Enthalpy
0,-0.000,"[La2Zr2O7, Li2O2, La2O3]",-0.745
1,0.126,"[La2O3, Li6Zr2O7, Li2O2]",-0.368
2,0.387,"[Li2O, Li6Zr2O7, La2O3]",-0.175
3,6.117,"[Zr3O, La2O3, Li2O]",20.215
4,6.204,"[Li2O, La2O3, Zr]",24.351
5,6.213,"[Li, La2O3, Zr]",46.095
6,6.491,"[La, Zr, Li]",75.305


,Chemical Potential,Phase Equilibria,Reaction Enthalpy
0,-0.000,[LiTi2(PO4)3],-0.000
1,3.920,"[LiTiPO5, P, TiO2]",19.602
2,3.942,"[TiO2, Li3PO4, P]",26.171
3,4.048,"[TiO2, TiP2, Li3PO4]",36.965
4,4.136,"[Li3PO4, TiP2, TiP]",42.480
5,4.758,"[Li3P7, TiP, Li3PO4]",44.594
6,4.804,"[LiP, TiP]",48.865
